In [1]:

import os
import pandas as pd


def get_input_csv(day: int):
    fp = os.path.join(os.getcwd(), "inputs", f"day{day}.csv")
    return pd.read_csv(fp, encoding="utf-8", header=None).rename(columns={0: "data"})


## Day 1

In [2]:
# Input:
data = get_input_csv(1)
data = list(data["data"].values)
data[:10]

[151, 152, 153, 158, 159, 163, 164, 162, 161, 167]

In [3]:
# Q1 solution:
answer = len(list(filter(lambda x: x[1] > data[x[0]], list(enumerate(data[1:])))))
answer

1564

In [4]:
# Q2 solution:
answer = len(list(filter(lambda ix: sum(data[ix-3: ix]) < sum(data[ix-2: ix+1]), range(3, len(data), 1))))
answer

1611

## Day 2

In [5]:
# Input:
data = get_input_csv(2)
data = list(data["data"].values)
data[:10]

['forward 3',
 'down 6',
 'forward 7',
 'down 4',
 'down 8',
 'down 6',
 'down 4',
 'down 6',
 'forward 7',
 'down 5']

In [6]:
# Q1 solution - needlessly compact 3 liner:
h, d = 0, 0
[exec({"f": "global h;h= h+int(i)", "d": "global d;d+=int(i)", "u": "global d;d-=int(i)"}[dr[0]]) for dr, i in map(str.split, data)]
answer = h*d
answer

1924923

In [7]:
# Q1 solution - slightly less needlessly compact 4 liner:
h, d = 0, 0
for dct in [{"f": {"h": int(i)}, "d": {"d": int(i)}, "u": {"d": -int(i)}}[s[0]] for s, i in map(str.split, data)]:
    h += dct.get("h", 0)
    d += dct.get("d", 0)
answer = h*d
answer

1924923

In [8]:
# Q1 solution - reduce:
from functools import reduce

iterable = [{"f": {"h": int(i)}, "d": {"d": int(i)}, "u": {"d": -int(i)}}[s[0]] for s, i in map(str.split, data)]
d = reduce(lambda x, y: x + y.get("d", 0), iterable, 0)
h = reduce(lambda x, y: x + y.get("h", 0), iterable, 0)
answer = h*d
answer

1924923

In [9]:
# Q1 solution - recursion:
def recursion(seq, h=0, d=0):
    if len(seq):
        s, i = seq[0].split()
        dct = {"f": {"h": int(i)}, "d": {"d": int(i)}, "u": {"d": -int(i)}}[s[0]]
        h += dct.get("h", 0)
        d += dct.get("d", 0)
        return recursion(seq[1:], h, d)
    else:
        return h * d
        
        
answer = recursion(data)
answer

1924923

In [10]:
# Q2 solution - basic brute force:
h, d, a = 0, 0, 0
for s, i in map(str.split, data):
    if s[0] == "f":
        h += int(i)
        d += a*int(i)
    elif s[0] == "d":
        a += int(i)
    elif s[0] == "u":
        a -= int(i)

answer = h*d
answer

1982495697